In [82]:
import os

In [84]:
files = os.listdir("catalogs/6sigma")

In [85]:
def create_transient_candidate_catalog(directory, filename, d_directory):
    
    import lsst.afw.table as afwTable

    
    catalog = afwTable.SourceCatalog.readFits(directory + "/" + filename)
    c_r  = catalog.copy()
    c_r.clear()
    for source in catalog:
        if source["classification_dipole"] == 6:
            c_r.append(source)
            
    c_r.writeFits(d_directory+"/"+filename)
    return filename.split('-')[0]

In [63]:
r = create_transient_candidate_catalog("catalogs/6sigma",files[0], "tc_catalogs")

In [86]:
import ipyparallel as ipyp

c = ipyp.Client()
v = c[:]


def f(a,b):
    def c(a, b):
        return a+b

    return c(a,b)
#res = v.map(lambda x,y: x+y, (1, 2, 3, 4), (2, 3, 4, 5))
res = v.map(f, [1, 2, 3, 4], [2, 3, 4, 5])

print res.get()

[3, 5, 7, 9]


In [87]:
directories = ["catalogs/6sigma"]* len(files)
d_directories = ["tc_catalogs/6sigma"]* len(files)
print len(directories)

3578


In [88]:
res = v.map(create_transient_candidate_catalog, directories, files, d_directories)

In [91]:
print res

<AsyncMapResult: create_transient_candidate_catalog:finished>


In [92]:
cats = res.get()
c_cats = list(set(cats))
print len(cats), len(c_cats)

3578 107


In [93]:
import lsst.afw.table as afwTable
import glob

for c_cat in c_cats:
    v_catalog = None
    
    c_files = glob.glob("tc_catalogs/6sigma/"+c_cat+"*.fits")
    for c_file in c_files:
        catalog = afwTable.SourceCatalog.readFits(c_file)
        
        if v_catalog is None:
            v_catalog = catalog.copy()
        else:
            v_catalog.extend(catalog)
    
    v_catalog.writeFits("vc_catalogs/6sigma/"+c_files[0].split("/")[-1].split("-")[0]+".fits")
    

In [23]:
import numpy as np

import lsst.daf.persistence as dafPersist
DATADIR="/renoir_data_02/jpreyes/lsst_data/CFHTLS_master/"

butler = dafPersist.Butler(DATADIR+"/output")
visits = butler.queryMetadata("deepDiff_differenceExp", format=['visit'],dataId={'filter':'r'})
visits = np.array(visits)
mask = (visits >= 836493) & (visits <= 860150)
visits =  visits[mask]
print visits

/renoir_data_00/fouchez/lsst/DM/stacks/lsstsw/2016-04-04/lsstsw/miniconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


[836493 836494 836495 836496 836497 836498 837002 837003 837004 837005
 837006 837007 837008 838853 838854 838855 838857 838864 838865 838866
 838867 838868 839308 839309 839310 839311 839312 844210 844211 844212
 844213 844214 844230 844489 844490 844491 844492 844493 844494 844495
 844496 844497 844498 844514 844515 844516 844517 844518 844845 844846
 844847 844848 844849 845345 845346 845347 845348 845349 849373 849374
 849375 849376 849377 849685 849686 849687 849688 849689 850177 850178
 850179 850180 850181 850586 850587 850588 850589 850590 851057 851058
 851059 851060 851061 852890 852891 852892 852893 852894 853232 853233
 853234 853235 853236 853539 853540 853541 853542 853543 853727 853728
 853729 853730 853731 858537 858538 858539 858540 858541 859608 859609
 859610 859611 859612 860146 860147 860148 860149 860150]


In [55]:
c_cat

836493

In [57]:
import lsst.afw.table as afwTable
import glob

for c_cat in visits:
    v_catalog = None
    #print "catalogs/6sigma/"+str(c_cat)+"*.fits"
    c_files = glob.glob("catalogs/6sigma/"+str(c_cat)+"*.fits")
    
    for c_file in c_files:
        catalog = afwTable.SourceCatalog.readFits(c_file)
        
        if v_catalog is None:
            v_catalog = catalog.copy()
        else:
            v_catalog.extend(catalog)
            
    if v_catalog is not None:
        v_catalog.writeFits("nc_catalogs/6sigma/"+c_files[0].split("/")[-1].split("-")[0]+".fits")
    

In [42]:
multi_matches = None
import lsst.afw.geom as afwGeom

for f in files:
    
    catalog = afwTable.SourceCatalog.readFits("vc_catalogs/3sigma/"+f)
    
    if multi_matches is None and len(catalog)>0:
        multi_matches = afwTable.MultiMatch(catalog[0].schema, {'visit':int}, radius=afwGeom.Angle(1./3600., afwGeom.degrees))
    if multi_matches is not None:    
        multi_matches.add(catalog, {'visit':int(f.split(".")[0])})
    


In [43]:
results = multi_matches.finish(removeAmbiguous=False)

In [44]:
def threshold_light_curves(light_curves, threshold):
    t_light_curves = [lc for lc in light_curves if len(lc) >= threshold]
    return t_light_curves


In [45]:
light_curves = []
i = 0
current = -1
while i < len(results):
    result = results[i]
    if current == -1 or current != result['object']:
        lc = [(result['visit'],result)]
        light_curves.append(lc)
        current = result['object']
    else:
        light_curves[-1].append((result['visit'],result))
    i+=1

t_light_curves = threshold_light_curves(light_curves, 10)

In [46]:
len(t_light_curves)

1109